# Aggregation
## definition
- the process of combining things or amounts into a single group or total
- the process of collecting information from several different websites, newspapers, databases (= large amounts of information stored in a computer system ), etc. and combining it in one place, or the result of this process:
- 각기 다를 자료들을 조합 해주는 작업

## 1. 종합
### 1.1. 단일열 기준 종합

In [3]:
import pandas as pd
import numpy as np

In [4]:
flights = pd.read_csv('C:/Users/user/Desktop/BigDataCourse/data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [7]:
flights['AIRLINE'].value_counts()

DL    10601
AA     8900
WN     8418
UA     7792
OO     6588
EV     5858
MQ     3471
US     1615
NK     1516
F9     1317
VX      993
AS      768
B6      543
HA      112
Name: AIRLINE, dtype: int64

In [4]:
flights.groupby('AIRLINE').agg({'DIVERTED':'sum'}).head() 
# DIVERTED의 평균을 구해줘 :mean
# DIVERTED의 횟수 sum
# AIRLINE 그룹화

,DIVERTED
AIRLINE,
AA,26
AS,0
B6,2
DL,24
EV,15


In [5]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head() 
# DELAY의 평균을 구해줘 :mean
# AIRLINE 그룹화

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [6]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head() 
# 데이터 프레임을 만들어

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [8]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

### 1.2. 복수열 기준 종합

In [9]:
flights.groupby(['AIRLINE','WEEKDAY'])['CANCELLED'].agg('sum').head()
#항공사들의 요일별 취소횟수

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
Name: CANCELLED, dtype: int64

In [10]:
flights.groupby(['AIRLINE','WEEKDAY']).agg({'CANCELLED':'sum'}).head(10)
#항공사들의 요일별 취소횟수

CANCELLED
AIRLINE WEEKDAY           
AA      1               41
        2                9
        3               16
        4               20
        5               18
        6               21
        7               29
AS      1                0
        2                0
        3                0

In [77]:
flights.groupby(['AIRLINE','WEEKDAY']).agg({'CANCELLED':['sum','mean'], 'DIVERTED':['sum','mean']}).head()

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [12]:
group = ['AIRLINE','WEEKDAY']
agg_dict = {'CANCELLED':['sum','mean'], 'DIVERTED':['sum','mean']}
flights.groupby(group).agg(agg_dict).head()

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786

In [13]:
group_cols = ['ORG_AIR','DEST_AIR'] # 출발지와 도착지를 기준으로 삼을 것
agg_dict = {'CANCELLED':['sum','mean','size'], 'AIR_TIME':['mean','var']}
flights.groupby(group_cols).agg(agg_dict).head(10)

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692
        AGS              0  0.0   83   28.819277   9.393770
        ALB              0  0.0   33  108.181818  41.903409
        ANC              0  0.0    2  438.500000  40.500000
        ASE              0  0.0    1  192.000000        NaN
        ATW              0  0.0   10  106.400000  61.377778

## 2. 종합 후 처리
### 2.1. MultiIndex 제거

In [14]:
air_info = flights.groupby(group_cols).agg(agg_dict)
air_info.head() # as_index=False는 행를 만들지 않고 바로 열로 만든다

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

In [15]:
# get_level_values : 요구 된 레벨의 인덱스를 돌려줍니다.
level0 = air_info.columns.get_level_values(0)
level0

Index(['CANCELLED', 'CANCELLED', 'CANCELLED', 'AIR_TIME', 'AIR_TIME'], dtype='object')

In [16]:
level1 = air_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'size', 'mean', 'var'], dtype='object')

In [17]:
air_info.columns = level0+'_'+level1
air_info.head()

CANCELLED_sum  CANCELLED_mean  CANCELLED_size  \
ORG_AIR DEST_AIR                                                  
ATL     ABE                   0             0.0              31   
        ABQ                   0             0.0              16   
        ABY                   0             0.0              19   
        ACY                   0             0.0               6   
        AEX                   0             0.0              40   

                  AIR_TIME_mean  AIR_TIME_var  
ORG_AIR DEST_AIR                               
ATL     ABE           96.387097     45.778495  
        ABQ          170.500000     87.866667  
        ABY           28.578947      6.590643  
        ACY           91.333333     11.466667  
        AEX           78.725000     47.332692

In [18]:
air_info.reset_index().head()
#index 를 reset 한다

,ORG_AIR,DEST_AIR,CANCELLED_sum,CANCELLED_mean,CANCELLED_size,AIR_TIME_mean,AIR_TIME_var
0,ATL,ABE,0,0.0,31,96.387097,45.778495
1,ATL,ABQ,0,0.0,16,170.500000,87.866667
2,ATL,ABY,0,0.0,19,28.578947,6.590643
3,ATL,ACY,0,0.0,6,91.333333,11.466667
4,ATL,AEX,0,0.0,40,78.725000,47.332692


In [19]:
air_info = flights.groupby(group_cols, as_index=False).agg(agg_dict)
air_info.head()

ORG_AIR DEST_AIR CANCELLED              AIR_TIME           
                         sum mean size        mean        var
0     ATL      ABE         0  0.0   31   96.387097  45.778495
1     ATL      ABQ         0  0.0   16  170.500000  87.866667
2     ATL      ABY         0  0.0   19   28.578947   6.590643
3     ATL      ACY         0  0.0    6   91.333333  11.466667
4     ATL      AEX         0  0.0   40   78.725000  47.332692

### 2.2. 사용자 정의함수 사용

In [99]:
def pct_between(s):
    return s.between(-100, 60).mean()

In [100]:
flights.groupby(['AIRLINE'])['DEP_DELAY','ARR_DELAY']

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [101]:
flights.groupby(['AIRLINE'])['DEP_DELAY','ARR_DELAY'].agg(pct_between)
#항공기 별

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,DEP_DELAY,ARR_DELAY
AIRLINE,,
AA,0.916854,0.915281
AS,0.972656,0.971354
B6,0.915285,0.907919
DL,0.959249,0.958211
EV,0.917890,0.912940
F9,0.913440,0.900532
HA,0.982143,0.973214
MQ,0.897148,0.889369
NK,0.868734,0.852902


In [93]:
flights.groupby(['ORG_AIR'])['DEP_DELAY','ARR_DELAY'].agg(pct_between)
#공항별

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,DEP_DELAY,ARR_DELAY
ORG_AIR,,
ATL,0.945453,0.944204
DEN,0.930169,0.929657
DFW,0.910687,0.908861
IAH,0.917427,0.914005
LAS,0.936054,0.934312
LAX,0.927152,0.925794
MSP,0.944868,0.944868
ORD,0.885990,0.880272
PHX,0.942863,0.941994


In [23]:
# pct.beetween(s)
# (-100, 60)
# 이거대신 사용자가 입력할수 있게 만들어보자

In [24]:
pct_between.__name__

'pct_between'

In [25]:
pct_between.__name__ = 'Non_Delay Ratio'

In [102]:
def pct_between(s, low, high):
    return s.between([min, max, sorted] ).mean()

In [103]:
flights.columns


Index(['MONTH', 'DAY', 'WEEKDAY', 'AIRLINE', 'ORG_AIR', 'DEST_AIR',
       'SCHED_DEP', 'DEP_DELAY', 'AIR_TIME', 'DIST', 'SCHED_ARR', 'ARR_DELAY',
       'DIVERTED', 'CANCELLED'],
      dtype='object')

In [104]:
flights.groupby(['AIRLINE'])['DEP_DELAY','ARR_DELAY'].agg(pct_between, -10, 10)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


AttributeError: 'DataFrame' object has no attribute 'between'

In [76]:
group = ['AIRLINE']
group2 = flight[[['AIRLINE','DEP_DELAY','ARR_DELAY']]
(group2).agg(pct_between, -10, 10)

SyntaxError: unexpected EOF while parsing (<ipython-input-76-8cb25ed7b7bd>, line 3)

In [73]:
flights[['DEP_DELAY','ARR_DELAY']]

,DEP_DELAY,ARR_DELAY
0,58.0,65.0
1,7.0,-13.0
2,36.0,35.0
3,7.0,-7.0
4,48.0,39.0
...,...,...
58487,5.0,-19.0
58488,13.0,4.0
58489,-6.0,-5.0
58490,39.0,34.0
